# Testing how to store models and weights

    this notebook is using CIFAR10 dataset 
    Keras version 1.0.2
    Theano version 0.9.0dev1


In [10]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np



In [14]:
# definition of variable
# data augmentation is set to false as it creates an error on my machine

batch_size = 32
nb_classes = 10
nb_epoch = 1       # only one epoch as I don't any accuracy, I only need a trained model 
data_augmentation = False

# input image dimensions
img_rows, img_cols = 32, 32

# the CIFAR10 images are RGB
img_channels = 3

In [3]:
# loading thje data
# the data, shuffled and split between train and test sets

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')



X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)



In [5]:
# definition of the model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



In [6]:
# some processing: change to float type and divide by 255

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [15]:
# again, no data augmentation used
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 634s - loss: 1.4225 - acc: 0.4843 - val_loss: 1.2105 - val_acc: 0.5636


## center part: saving model and weights

In [17]:
# the model is aved in the JSON string, eventually , this string needs to be saved in a file

json_string = model.to_json()

In [36]:
json_string

'{"class_name": "Sequential", "config": [{"class_name": "Convolution2D", "config": {"b_regularizer": null, "W_constraint": null, "b_constraint": null, "name": "convolution2d_1", "activity_regularizer": null, "trainable": true, "dim_ordering": "th", "nb_col": 3, "subsample": [1, 1], "init": "glorot_uniform", "bias": true, "nb_filter": 32, "input_dtype": "float32", "border_mode": "same", "batch_input_shape": [null, 3, 32, 32], "W_regularizer": null, "activation": "linear", "nb_row": 3}}, {"class_name": "Activation", "config": {"activation": "relu", "trainable": true, "name": "activation_1"}}, {"class_name": "Convolution2D", "config": {"W_constraint": null, "b_constraint": null, "name": "convolution2d_2", "activity_regularizer": null, "trainable": true, "dim_ordering": "th", "nb_col": 3, "subsample": [1, 1], "init": "glorot_uniform", "bias": true, "nb_filter": 32, "border_mode": "valid", "b_regularizer": null, "W_regularizer": null, "activation": "linear", "nb_row": 3}}, {"class_name": "A

In [19]:
# save the weights in model_weights file (binary)

model.save_weights('./model_weights')

### simulate a dataset without labels for which we need a predict

In [21]:
# create a set of 10 data records (copied from the training set)

X_test = X_train[10:20]

In [37]:
y_test = y_train[10:20]

In [38]:
y_test

array([[4],
       [7],
       [7],
       [2],
       [9],
       [9],
       [9],
       [3],
       [2],
       [6]], dtype=uint8)

In [22]:
# predict method of model

classes = model.predict_classes(X_test, batch_size=10)
proba = model.predict_proba(X_test, batch_size=10)

10/10 [==============================] - 0s


In [23]:
# result as categories
classes

array([4, 7, 4, 0, 9, 8, 9, 3, 6, 6])

In [24]:
# result as probabilties, the highest probability for each item is chosen for the classification

proba

array([[  9.25784989e-05,   3.54626764e-06,   5.67369871e-02,
          1.48985386e-02,   7.32406616e-01,   3.44633800e-03,
          1.90707698e-01,   1.68967794e-03,   9.36454398e-06,
          8.68152711e-06],
       [  1.22841168e-02,   6.62187487e-03,   3.16324346e-02,
          2.28643995e-02,   1.89307630e-01,   3.49498689e-02,
          6.40458008e-03,   6.81697726e-01,   1.54588430e-03,
          1.26915043e-02],
       [  2.50246257e-01,   7.98121002e-03,   1.10884674e-01,
          2.75632809e-03,   3.18268746e-01,   5.70206856e-03,
          1.80390792e-03,   2.86502570e-01,   2.10575131e-03,
          1.37484809e-02],
       [  2.78373241e-01,   2.64492244e-01,   1.23803265e-01,
          3.83267775e-02,   5.63055873e-02,   1.53252091e-02,
          1.16270510e-02,   3.28190550e-02,   1.23840779e-01,
          5.50868064e-02],
       [  4.21216451e-02,   1.01479463e-01,   2.81665903e-02,
          3.88136059e-02,   2.77727880e-02,   1.97520517e-02,
          3.34110819e-02

### loading model and weights back

In [27]:
# load back model

from keras.models import model_from_json

model_reloaded = model_from_json(json_string)

In [29]:
# load back weights

model_reloaded.load_weights('./model_weights')

In [30]:
# compile model as before

model_reloaded.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [31]:
# predict and check classification and probabilities are the same

classes = model_reloaded.predict_classes(X_test, batch_size=10)
proba = model_reloaded.predict_proba(X_test, batch_size=10)

10/10 [==============================] - 0s


In [32]:
classes

array([4, 7, 4, 0, 9, 8, 9, 3, 6, 6])

In [33]:
proba

array([[  9.25784989e-05,   3.54626764e-06,   5.67369871e-02,
          1.48985386e-02,   7.32406616e-01,   3.44633800e-03,
          1.90707698e-01,   1.68967794e-03,   9.36454398e-06,
          8.68152711e-06],
       [  1.22841168e-02,   6.62187487e-03,   3.16324346e-02,
          2.28643995e-02,   1.89307630e-01,   3.49498689e-02,
          6.40458008e-03,   6.81697726e-01,   1.54588430e-03,
          1.26915043e-02],
       [  2.50246257e-01,   7.98121002e-03,   1.10884674e-01,
          2.75632809e-03,   3.18268746e-01,   5.70206856e-03,
          1.80390792e-03,   2.86502570e-01,   2.10575131e-03,
          1.37484809e-02],
       [  2.78373241e-01,   2.64492244e-01,   1.23803265e-01,
          3.83267775e-02,   5.63055873e-02,   1.53252091e-02,
          1.16270510e-02,   3.28190550e-02,   1.23840779e-01,
          5.50868064e-02],
       [  4.21216451e-02,   1.01479463e-01,   2.81665903e-02,
          3.88136059e-02,   2.77727880e-02,   1.97520517e-02,
          3.34110819e-02